In [ ]:
# Load Azure Libaries
from azureml.core import Workspace, Dataset, Datastore, Experiment
from azureml.core.compute import ComputeTarget
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails

# Load Non-Azure Libraries
import pandas as pd
import numpy as np

In [ ]:
# Check the base version of AzureMl
import azureml.core
print("You are currently using version " + azureml.core.VERSION + " of the Azure ML SDK")

In [ ]:
# Retrieve your workspace by name by filling in the lower case values between double quotes
ws = Workspace.from_config()

In [ ]:
# Retrieve your Compute Target for Running AutoML Remotely
compute_name = 'compute-cluster'
compute_target = ComputeTarget(ws, compute_name)

In [ ]:
# Retrieve your Datastore by name by filling in the lower case values between double quotes
datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws, my_datastore_name)

In [ ]:
# If you have already made the Diabetes dataset, retrieve it
dataset_name = "Diabetes Sample"
dataset = Dataset.get_by_name(ws, dataset_name, version='latest')

In [ ]:
# If you have not already made the Diabetes dataset, register it

# import the Diabetes open dataset
from azureml.opendatasets import Diabetes

# Retrieve the Diabetes Dataset
diabetes = Diabetes.get_tabular_dataset()

# Register the Dataset
# Set the dataset name, description and tags as you wish
data_registered = diabetes.register(workspace=ws,
                                              name='Diabetes Sample', 
                                              description='Sample Diabetes Data from Azure Open Datasets', 
                                              tags= {'fileType': 'Tabular', "project" : "Diabetes"}, 
                                              create_new_version=True)

In [ ]:
# View your dataset by converting to pandas
dataset.take(10).to_pandas_dataframe()

In [ ]:
# Turn your dataset into a Pandas dataframe
dfRaw = dataset.to_pandas_dataframe()

# Drop the Age and Sex Columns
df = dfRaw.drop(['AGE','SEX'], axis=1)

In [ ]:
# Register a new Dataset using the Dataframe with the Age and Sex columns dropped
Dataset.Tabular.register_pandas_dataframe(df, datastore,
                                          "Diabetes Sample Age/Sex Dropped")

In [ ]:
# Create a name for your AutoML Diabetes Regression Experiment
experiment_name = 'Diabetes-Sample-Regression'

# Set the Experiment to your Workspace
exp = Experiment(workspace=ws, name=experiment_name)

In [ ]:
# Set variables for your AutoML Run
target_column = 'Y'
task = 'regression'
primary_metric = 'normalized_root_mean_squared_error'
featurization = 'auto'

In [ ]:
# Set your AutoML Configuration
config = AutoMLConfig(task=task,
                     primary_metric=primary_metric,
                     featurization=featurization,
                     compute_target=compute_target,
                     training_data=dataset,
                     label_column_name=target_column,
                     experiment_timeout_minutes=15,
                     enable_early_stopping=True,
                     n_cross_validations=5,
                     model_explainability=True)

In [ ]:
# Train your AutoML Diabetes Regression Model
AutoML_run = exp.submit(config, show_output = True)
RunDetails(AutoML_run).show()

In [ ]:
# Register your Trained AutoML Model to your AMLS Workspace
description = 'Best AutoML Regression Run using Diabetes Sample Data. This model requires the Age and Sex Columns.' 
tags = {'project' : "Diabetes", "creator" : "your name"} 
model_name = 'Diabetes-AllData-Regression-AutoML' 

AutoML_run.register_model(model_name=model_name,description=description,tags=tags)

In [ ]:
# Register your Trained AutoML Model to your AMLS Workspace using R2 Score instead of Normalized Root Mean Squared Error
description = 'Best AutoML Regression Run using Diabetes Sample Data. This model requires the Age and Sex Columns.' 
tags = {'project' : "Diabetes", "creator" : "your name", "metric" : "R2"} 
model_name = 'Diabetes-AllData-Regression-AutoML-R2' 
AutoML_run.register_model(model_name=model_name, description=description, tags=tags, metric = 'r2_score')
